In [2]:
import os
# from getpass import getpass
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers.pipelines.pt_utils import KeyDataset
#from wandb.integration.langchain import WandbTracer
import wandb
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.evaluation.qa import QAEvalChain
import torch
from datasets import load_dataset
import argparse
import json
from sklearn.metrics import precision_recall_fscore_support
import pandas as pd
import math
import re
from time import time,sleep
import string
import collections

# from tqdm.notebook import tqdm
from tqdm import tqdm

from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import AIMessage, HumanMessage, SystemMessage

In [4]:
data = pd.read_csv('../data/lake-erie-reddit_all.csv')
texts = data['body'].to_list()

In [4]:
# put your api key here
API_KEY=None

In [7]:
max_length = 2048

llm = ChatOpenAI(
    # model_name='gpt-4', 
    model_name='gpt-3.5-turbo', 
    openai_api_key=API_KEY, 
    temperature=0,
    max_retries=12, 
    request_timeout=600)

template = "{text}"
prompt = PromptTemplate(template=template, input_variables=["text"])
llm_chain = LLMChain(prompt=prompt,llm=llm)

outputs = []

for i, text in enumerate(tqdm(texts)):
    # designed prompt for stance detection
    ppt_template = f"""Context: Lake Erie and surrounding water bodies are subject to various policies aimed at protecting their water quality, addressing issues from pollution spills to toxic algal blooms. Public opinion on the effectiveness of these policies is crucial for ongoing environmental efforts.

Task: Read the following comment carefully. Determine if the comment expresses a belief regarding the effectiveness of current policies aimed at protecting Lake Erie's water quality.

If the comment suggests that the policies are making a positive impact, answer 'Yes'.
If the comment suggests that the policies are not making a positive impact, answer 'No'.
If the comment does not express a stance on the effectiveness of these policies or is unrelated to the policy effectiveness, answer 'None'.
Please do not provide any reasoning for your answer. Simply state 'Yes', 'No', or 'None' based on the content of the comment.

Comment: {text}

Answer:"""

    try:
        output = llm_chain.run(ppt_template)
    except Exception as e:
        print(f"Error processing text at index {i}: {e}")
        output = None
 
    outputs.append([text, output])
    
    if (i + 1) % 1000 == 0:
        file_path = f'stance_detection_gpt3.5_{i + 1}.json'
        with open(file_path, 'w') as file:
            json.dump(outputs, file, indent=4)
        print(f'Saved outputs up to index {i} to {file_path}')

        
final_path = f'stance_detection_gpt3.5.json'
with open(final_path, 'w') as file:
    json.dump(outputs, file, indent=4)
print(f'Saved final batch of outputs to {final_path}')

 20%|██        | 1000/4915 [04:59<22:58,  2.84it/s] 

Saved outputs up to index 999 to stance_detection_gpt3.5_1000.json


 41%|████      | 2000/4915 [11:17<15:16,  3.18it/s]   

Saved outputs up to index 1999 to stance_detection_gpt3.5_2000.json


 61%|██████    | 3000/4915 [16:10<12:03,  2.65it/s]

Saved outputs up to index 2999 to stance_detection_gpt3.5_3000.json


 81%|████████▏ | 4000/4915 [21:05<04:14,  3.59it/s]

Saved outputs up to index 3999 to stance_detection_gpt3.5_4000.json


100%|██████████| 4915/4915 [25:36<00:00,  3.20it/s]

Saved final batch of outputs to stance_detection_gpt3.5.json
